In [10]:
# Tambahkan ini di bagian atas script Anda (sebagai reminder):
import glob 

def draw_nutrition_box(original_image_path, nutrition_report_lines):
    """Memuat gambar, menggambar kotak, dan menulis teks nutrisi."""
    
    output_filename_base = os.path.splitext(os.path.basename(original_image_path))[0]
    
    # 1. Cari folder output YOLO terbaru (e.g., runs/detect/predict3)
    latest_predict_dirs = sorted(glob.glob('runs/detect/predict*'))
    if not latest_predict_dirs:
        print("ERROR: Tidak ada folder output deteksi YOLO yang ditemukan.")
        return None
        
    latest_predict_dir = latest_predict_dirs[-1]

    # 2. Cari file yang namanya cocok di folder output terbaru (mengabaikan ekstensi)
    search_path = os.path.join(latest_predict_dir, f"{output_filename_base}.*")
    found_files = glob.glob(search_path)
    
    if not found_files:
        # Coba lagi dengan case-sensitive files (jika ada)
        print(f"ERROR: File deteksi untuk '{output_filename_base}' tidak ditemukan di {latest_predict_dir}")
        return None

    yolo_detected_path = found_files[0] # Ambil file pertama yang cocok
    
    print(f"File deteksi ditemukan di: {yolo_detected_path}")

    # --- OpenCV Drawing ---
    img = cv2.imread(yolo_detected_path)
    
    # Safety Check: Pastikan gambar dimuat
    if img is None:
        print(f"ERROR: Gagal memuat gambar OpenCV dari {yolo_detected_path}")
        return None

    H, W, _ = img.shape
    
    # ... (lanjutan konstanta drawing)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.7
    line_spacing = 30
    box_padding = 10
    
    # Hitung posisi kotak (pojok kanan atas)
    num_lines = len(nutrition_report_lines)
    box_width = 350
    box_height = (num_lines * line_spacing) + (box_padding * 2)
    
    start_x = W - box_width - 10
    start_y = 10
    end_x = W - 10
    end_y = start_y + box_height
    
    # Gambar latar belakang (putih)
    cv2.rectangle(img, (start_x, start_y), (end_x, end_y), (255, 255, 255), -1)
    # Gambar batas (hitam)
    cv2.rectangle(img, (start_x, start_y), (end_x, end_y), (0, 0, 0), 2)
    
    # Tulis teks per baris
    for i, line in enumerate(nutrition_report_lines):
        text_y = start_y + box_padding + ((i + 1) * line_spacing) - 5
        # Tulis teks (hitam)
        cv2.putText(img, line, (start_x + box_padding, text_y), font, font_scale, (0, 0, 0), 2, cv2.LINE_AA)

    # Simpan gambar akhir (di folder runs/detect/final_output)
    final_output_filename = 'nutri_' + os.path.basename(yolo_detected_path)
    final_output_path = os.path.join('runs/detect/final_output', final_output_filename)
    os.makedirs('runs/detect/final_output', exist_ok=True)
    cv2.imwrite(final_output_path, img)
    
    return final_output_path

# **Integrasi dan Perhitungan Nutrisi**

In [7]:
import pandas as pd
import yaml 
import os
import cv2 
from ultralytics import YOLO 
from collections import Counter
import random 
import glob 
import numpy as np 
import itertools 

# =======================================================
# 1. KONFIGURASI PATH & DATA (WAJIB DEFINISI DULU)
# =======================================================
# Semua variabel path harus didefinisikan di sini!
MODEL_PATH = "indonesian_food_yolov8/yolov8m_extended_ft/weights/best.pt"
DATA_YAML_PATH = r"C:\Users\macin\OneDrive - Universitas Pelita Harapan\Documents\NeuralNetwork\Food Recognation\food-recognition-AML\Food Dataset\data.yaml"
NUTRITION_CSV_PATH = r'C:\Users\macin\OneDrive - Universitas Pelita Harapan\Documents\NeuralNetwork\Nutrition.csv' 

# Load konfigurasi dataset untuk mendapatkan nama kelas (yang final)
# Bagian ini kini aman karena DATA_YAML_PATH sudah didefinisikan di atas.
with open(DATA_YAML_PATH, "r") as f:
    data_config = yaml.safe_load(f)
    final_class_names = data_config['names']

# Tentukan direktori gambar uji 
TEST_IMAGE_DIR = os.path.join(os.path.dirname(DATA_YAML_PATH), 'test', 'images')

# =======================================================
# 2. FUNGSI UTAMA: DETEKSI DAN KALKULASI
# =======================================================

def analyze_food_image(model_path, image_path, df_nutrition, class_names):
    """Memuat model, mendeteksi objek, dan menghitung nutrisi.
       Sekarang mengembalikan string laporan, DataFrame, dan output_dir."""
    
    try:
        model = YOLO(model_path)
    except Exception as e:
        print(f"ERROR: Gagal memuat model YOLO dari {model_path}. Pesan: {e}")
        return "ERROR: Model Gagal Dimuat.", None, None

    # 2.1 Deteksi Objek (Simpan gambar deteksi)
    print(f"-> Memproses gambar: {os.path.basename(image_path)}")
    # save=True diatur untuk menyimpan gambar ber-BBox
    results = model.predict(source=image_path, conf=0.25, iou=0.7, verbose=False, save=True, project='runs/detect', name='predict', exist_ok=True) 
    
    # Dapatkan folder output spesifik yang baru saja dibuat oleh YOLO
    output_save_dir = results[0].save_dir # <--- KUNCI PERBAIKAN: Mengambil folder output yang tepat

    # 2.2 Hitung Jumlah (Counts)
    detection_counts = Counter()
    if results and results[0].boxes and len(results[0].boxes.cls) > 0:
        for det in results[0].boxes:
            # FIX WARNING: Menggunakan .item() untuk konversi aman dari array NumPy scalar
            class_id = int(det.cls.cpu().numpy().item()) 
            food_name = class_names[class_id]
            detection_counts[food_name] += 1
    
    if not detection_counts:
        return "Tidak ada makanan yang terdeteksi.", None, None

    # 2.3 Kalkulasi Nutrisi
    total_nutrition = {'energi_kcal': 0, 'protein_g': 0, 'lemak_g': 0, 'karbohidrat_g': 0}
    results_list = []
    # ... (lanjutan kalkulasi nutrisi)

    for food_name, count in detection_counts.items():
        if food_name in df_nutrition.index:
            nut_total = df_nutrition.loc[food_name] * count
            results_list.append({
                'Makanan': food_name, 'Jumlah': count, 
                'Kalori (kcal)': nut_total['energi_kcal'], 'Protein (g)': nut_total['protein_g'], 
                'Lemak (g)': nut_total['lemak_g'], 'Karbohidrat (g)': nut_total['karbohidrat_g']
            })
            for key in total_nutrition.keys():
                total_nutrition[key] += nut_total[key]

    # 2.4 FORMAT UNTUK DISPLAY DI GAMBAR (RINGKAS)
    df_results = pd.DataFrame(results_list)
    report_lines = []
    report_lines.append(f"TOTAL KALORI: {total_nutrition['energi_kcal']:.0f} kcal")
    report_lines.append(f"Protein: {total_nutrition['protein_g']:.1f}g | Lemak: {total_nutrition['lemak_g']:.1f}g")
    report_lines.append(f"Karbohidrat: {total_nutrition['karbohidrat_g']:.1f}g")

    return "\n".join(report_lines), df_results, output_save_dir # <-- MENGEMBALIKAN FOLDER OUTPUT


def draw_nutrition_box(original_image_path, nutrition_report_lines, yolo_output_dir):
    """Memuat gambar, menggambar kotak, dan menulis teks nutrisi."""
    
    # 1. Tentukan Path Gambar Output YOLO
    input_filename = os.path.basename(original_image_path)
    output_filename_base = os.path.splitext(input_filename)[0]
    
    # Cari file yang namanya cocok di folder output yang spesifik
    # Mencari file yang namanya sama DENGAN EKSTENSI APAPUN di folder output SPESIFIK yang baru dibuat
    search_path = os.path.join(yolo_output_dir, f"{output_filename_base}.*")
    found_files = glob.glob(search_path)
    
    if not found_files:
        # PENTING: Jika YOLO memotong nama file karena terlalu panjang, kita coba cari file yang Paling Mirip
        # Namun, kita asumsikan output file tetap sama namanya dengan input file (meski beda ekstensi)
        print(f"ERROR: File deteksi untuk '{input_filename}' tidak ditemukan di lokasi: {yolo_output_dir}")
        return None

    yolo_detected_path = found_files[0] # Ambil file pertama yang cocok

    # --- OpenCV Drawing ---
    img = cv2.imread(yolo_detected_path)
    
    if img is None:
        print(f"ERROR: Gagal memuat gambar OpenCV dari {yolo_detected_path}")
        return None
    
    H, W, _ = img.shape
    
    # ... (lanjutan logika drawing tetap sama)
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.7
    line_spacing = 30
    box_padding = 10
    
    num_lines = len(nutrition_report_lines)
    box_width = 350
    box_height = (num_lines * line_spacing) + (box_padding * 2)
    
    start_x = W - box_width - 10
    start_y = 10
    end_x = W - 10
    end_y = start_y + box_height
    
    # Gambar latar belakang (putih) dan batas
    cv2.rectangle(img, (start_x, start_y), (end_x, end_y), (255, 255, 255), -1)
    cv2.rectangle(img, (start_x, start_y), (end_x, end_y), (0, 0, 0), 2)
    
    # Tulis teks per baris
    for i, line in enumerate(nutrition_report_lines):
        text_y = start_y + box_padding + ((i + 1) * line_spacing) - 5
        cv2.putText(img, line, (start_x + box_padding, text_y), font, font_scale, (0, 0, 0), 2, cv2.LINE_AA)

    # Simpan gambar akhir (di folder runs/detect/final_output)
    final_output_filename = 'nutri_' + os.path.basename(yolo_detected_path)
    final_output_path = os.path.join('runs/detect/final_output', final_output_filename)
    os.makedirs('runs/detect/final_output', exist_ok=True)
    cv2.imwrite(final_output_path, img)
    
    return final_output_path


# =======================================================
# 3. PERSIAPAN DATA NUTRISI & JALANKAN PROSES
# =======================================================
try:
    # ... (muat data nutrisi, ambil gambar random)
    df_nutrition = pd.read_csv(NUTRITION_CSV_PATH)
    # ... (pembersihan df_nutrition)
    df_nutrition['nama_makanan'] = df_nutrition['nama_makanan'].str.title()
    df_nutrition['nama_makanan'] = df_nutrition['nama_makanan'].replace('Nasi Goreng', 'Nasi Goreng')
    df_nutrition['nama_makanan'] = df_nutrition['nama_makanan'].replace('Ikan Goreng', 'Ikan Goreng')
    columns_to_keep = ['nama_makanan', 'energi_kcal', 'protein_g', 'lemak_g', 'karbohidrat_g']
    df_nutrition = df_nutrition[columns_to_keep].set_index('nama_makanan')

    # --- Ambil Gambar Random ---
    image_files = []
    for ext in ['*.jpg', '*.jpeg', '*.png']:
        image_files.extend(glob.glob(os.path.join(TEST_IMAGE_DIR, ext)))
    
    if not image_files:
        print(f"ERROR: Tidak ditemukan gambar di direktori uji: {TEST_IMAGE_DIR}")
        print("MOHON PERIKSA PATH DAN STATUS FILE DI ONEDRIVE!")
        exit() 

    random_image_path = random.choice(image_files)

    # --- Jalankan Deteksi & Kalkulasi ---
    # Perhatikan: Sekarang menerima 3 nilai
    nutrition_report_string, df_detail, yolo_output_dir = analyze_food_image(MODEL_PATH, random_image_path, df_nutrition, data_config['names'])
    
    # --- Gambar dan Simpan Hasil Akhir ---
    if isinstance(nutrition_report_string, str) and nutrition_report_string.startswith("Tidak ada"):
        final_image_path = "Tidak ada gambar visualisasi karena tidak ada deteksi."
        print(f"\n\n{nutrition_report_string}")
    else:
        # Meneruskan yolo_output_dir yang tepat ke fungsi drawing
        report_lines = nutrition_report_string.split('\n')
        final_image_path = draw_nutrition_box(random_image_path, report_lines, yolo_output_dir)
    
    # --- Cetak Hasil ---
    print("\n\n=======================================================")
    print("✅ LAPORAN ANALISIS NUTRISI VISUAL DAN DETAIL")
    print("=======================================================")
    print(f"Gambar Uji: {os.path.basename(random_image_path)}")
    print(f"Hasil Visual: Gambar disimpan di: {final_image_path}")
    
    if final_image_path != "Tidak ada gambar visualisasi karena tidak ada deteksi.":
        print("\n--- Detail Nutrisi Lengkap ---")
        print(df_detail.to_string(index=False))
        print("\n=======================================================")

except Exception as e:
    print(f"ERROR Fatal: Gagal memuat, memproses, atau memvisualisasikan data. Pesan: {e}")
    exit()

-> Memproses gambar: Nasi-Goreng_154_jpg.rf.e227adcad0d43fe5458d24ed41da39a0.jpg
Results saved to runs\detect\predict


✅ LAPORAN ANALISIS NUTRISI VISUAL DAN DETAIL
Gambar Uji: Nasi-Goreng_154_jpg.rf.e227adcad0d43fe5458d24ed41da39a0.jpg
Hasil Visual: Gambar disimpan di: runs/detect/final_output\nutri_Nasi-Goreng_154_jpg.rf.e227adcad0d43fe5458d24ed41da39a0.jpg

--- Detail Nutrisi Lengkap ---
    Makanan  Jumlah  Kalori (kcal)  Protein (g)  Lemak (g)  Karbohidrat (g)
Nasi Goreng       1          168.0          6.3       6.23            21.06

